## ЗАДАНИЕ ДЛЯ ПРОГРАММИРУЮЩИХ

**Ссылка**, на источник текста

In [1]:
DATA_URL = "http://az.lib.ru/t/tolstoj_a_k/text_0180.shtml"

Устанавливаем библиотеки

In [2]:
! pip install -q PyYaml==5.3.1
! pip install -q rnnmorph==0.4.0
! pip install -q keras==2.1.4
! pip install -q h5py==2.7.0

     |████████████████████████████████| 269 kB 5.2 MB/s 
     |████████████████████████████████| 10.5 MB 6.2 MB/s 
     |████████████████████████████████| 55 kB 2.8 MB/s 
     |████████████████████████████████| 8.2 MB 47.2 MB/s 
     |████████████████████████████████| 462 kB 44.0 MB/s 
     |████████████████████████████████| 322 kB 5.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires keras<2.9,>=2.8.0rc0, but you have keras 2.1.4 which is incompatible.
     |████████████████████████████████| 256 kB 5.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires h5py>=2.9.0, but you have h5py 2.7.0 which is incompatible.
tensorflow 2.8.0 requires keras<2.9,>=2.8.0rc0, but you have keras 

Создаём объект морфологического анализатора `RNNMorph`

In [3]:
%tensorflow_version 1.x
import warnings
warnings.filterwarnings('ignore')

from rnnmorph.predictor import RNNMorphPredictor
predictor = RNNMorphPredictor(language="ru")

TensorFlow 1.x selected.


Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Скачиваем текст, по которому будет дано задание, с помощью `urllib`

In [4]:
import urllib.request

opener = urllib.request.URLopener({})
resource = opener.open(DATA_URL)
raw_text = resource.read().decode(resource.headers.get_content_charset()) #Текс с html тегами

In [5]:
raw_text[:200]

'<html>\r\n<head>\r\n<title>Lib.ru/Классика: Толстой Алексей Константинович. Семья вурдалака</title>\r\n</head>\r\n\r\n<body>\r\n\r\n\r\n<center>\r\n\r\n<h2><a href=/t/tolstoj_a_k/>Толстой Алексей Константинович</a><br>\r\n'

Как видно, текст содержит html теги, от которых нужно избавиться. Выбрасываем из текста HTML-теги с помощью библиотеки Beatiful soap

In [6]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(raw_text, features="html.parser")

# kill all script and style elements
for script in soup(["script", "style"]):
    script.extract()    # rip it out

# get text
cleaned_text = soup.get_text()

In [7]:
cleaned_text[:200]

'\n\nLib.ru/Классика: Толстой Алексей Константинович. Семья вурдалака\n\n\n\nТолстой Алексей Константинович\r\nСемья вурдалака\n\n\nLib.ru/Классика:\n\r\n\n\n[Регистрация]\n \n\r\n\r\n\r\n[Найти] \r\n[Рейтинги]\r\n[Обсуждения]\r\n['

In [61]:
cleaned_text = ""
f = open('text.txt')
for line in f:
  cleaned_text += line

С помощью библиотеки [NLTK](https://nltk.org/) разбиваем текст на предложения и токены.

In [62]:
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')

tokenized_sentences = [word_tokenize(sentence) for sentence in sent_tokenize(cleaned_text)]
"A total of %d 'sentences'" % len(tokenized_sentences)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


"A total of 1423 'sentences'"

## Задание 1
С помощью метода `str.isalpha` из стандартной библиотеки Python модифицируйте нижеследующий код так, чтобы в predictions остались только буквенные токены.

In [63]:
tqdmCollection = tqdm(predictor.predict_sentences(sentences=tokenized_sentences), "sentences")

sentences:   0%|          | 0/1423 [00:00<?, ?it/s]

In [64]:
from tqdm import tqdm
#[pred.normal_form for pred in sent] 
predictions = []
for sent in tqdmCollection:
  for word in sent:
    if word.normal_form.isalpha():
      predictions.append(word.normal_form)


sentences: 100%|██████████| 1423/1423 [00:00<00:00, 48486.90it/s]


Проверьте себя. Должны получиться следующие значения:

*   Предложений: 577 (возможны расхождения в несколько предложений)
*   Токенов: примерно 8621 (возможны расхождения в некоторое количество токенов)

In [66]:
len(predictions)

20377

In [67]:
non_uniq_tokens = [word for sentence in predictions for word in sentence]
len(non_uniq_tokens) 


105738

Для продолжения работы над заданием числа должны быть близки к указанным

## Задание 2

Используя `non_uniq_tokens`, стоп-слова для русского языка из библиотеки nltk (`nltk.corpus.stopwords`) и `nltk.FreqDist`, вычислите, **какую долю среди 100 самых частотных** токенов в произведении занимают токены, **не относящиеся** к стоп словам. 

**Например**, если среди 100 самых частотных слов встречается 25 слов, входящих в стоп лист, значит не входят в стоп лист 75 слов, и их доля составит 0.75. 

**Не бойтесь использовать документацию NLTK и тьюториалы.**

In [68]:
import nltk
from nltk import FreqDist
from nltk.corpus import stopwords
nltk.download("stopwords")
STOPWORDS = set(stopwords.words("russian"))
stopwords.words("russian")[:5] #Пример стоп слов

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['и', 'в', 'во', 'не', 'что']

In [73]:
dictPredict = nltk.FreqDist(predictions)
listWordsPrediction = []

count = 0
for i,j in (dict(sorted(dictPredict.items(), key=lambda item: item[1], reverse=True))).items():
  listWordsPrediction.append(i)
  count+=1
  if(count == 40):
    break


In [74]:
count = 0
for word in listWordsPrediction:
  if(word in STOPWORDS):
    count+=1
count/100


0.29

Проверьте себя: должно получиться 0.49 (допустимо небольшое расхождение)

## Задание 3
Вычислите, сколько токенов встречается в тексте **строго больше** 50 раз.

In [71]:
cc = 0
for  key,value in dict(sorted(dictPredict.items(), key=lambda item: item[1], reverse=True)).items():
  if value < 21:
    break
  cc+=1



In [72]:
cc

131

Проверьте себя: должно получиться значение 22 (возможно небольшое расхождение)
